In [1]:
import pandas as pd
import re
import requests
import urllib
import nltk
from nltk.corpus import stopwords
import pymorphy2
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

#nltk.download('stopwords')
#nltk.download('punkt')

In [2]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [3]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [4]:
def clean_string(string):
    dirt_patterns = ['\'','\\n','\n','\\\\/S+','\\\\','/\S+','\(.*\)']
    for pattern in dirt_patterns:
        string = re.sub(pattern,'',string)
    return string

In [5]:
def search_theme(string):
    res = re.findall('«.*»',string)
    return res[0] if len(res)>0 else ""

In [6]:
def search_address(string):
    res = re.findall('по адресу\s?(.*$)',string)
    return res[0] if len(res)>0 else ""

In [7]:
def preprocessingText(line):
    line = line.lower()
    line = re.sub(r'\d+','',line,flags=re.UNICODE)#delete numbers
    line = re.sub(r'[^\w\s]',' ',line,flags=re.UNICODE)
    tl = nltk.word_tokenize(line) #tl - tokenize line
    nft = [morph.parse(token)[0].normal_form for token in tl if len(token)>1]#nft - normal form token
    clean_words_list = [normalToken for normalToken in nft if normalToken not in stop_words and len(normalToken)>1]#преобразованный список
    return ' '.join(clean_words_list)

In [8]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [9]:
df["theme"] = df['Тема обращения'].apply(search_theme)

In [10]:
df["address"] = df['Тема обращения'].apply(search_address)

In [11]:
df['Obr_clean'] = df['Суть обращения'].apply(preprocessingText)

In [12]:
df.head(2)

,Тема обращения,Суть обращения,Чиновник,Должность,Дата подачи заявления,theme,address,Obr_clean
0,Сообщение о проблеме «Неубранная городская тер...,После проведения работ на кабельной канализаци...,Ларин А.С.,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57,«Неубранная городская территория»,Снежная улица д.6,проведение работа кабельный канализация мгтс з...
1,Сообщение о проблеме «Неубранная городская тер...,Очистите опору освещения. Приведите в надлежащ...,Кучма А.А.,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54,«Неубранная городская территория»,Снежная улица д.24Д,очистить опор освещение привести надлежащий со...


In [13]:
df.to_csv('clean_frame.csv',sep='|',encoding='utf-8')

Вторая часть после загрузки и обработки данных

In [14]:
n_df = pd.read_csv('clean_frame.csv', sep='|', encoding='utf-8')

In [15]:
unique_addresses = n_df['address'].unique().tolist()

In [16]:
mapbox_token = "pk.eyJ1IjoidmFkaW05NiIsImEiOiJja25nMGowN2QyNnFlMnFtdTkzYjBjejdkIn0.18dpTc8Vgja2gPDUYgrSkw"

In [17]:
def search_district(address):
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{0}.json?access_token={1}".format(urllib.parse.quote("Москва "+str(address)), mapbox_token)
    response = requests.get(url)
    district = ""
    try:
        district = response.json()['features'][0]['place_name']
    except Exception as error:
        print("Возникла ошибка-{0}, адрес - {1}".format(str(error),str(address)) )
    finally:
        return district

In [ ]:
with open('unique_addresses_frame.csv','w',encoding='utf-8') as file:
    file.write('Адрес|Полный ответ\n')
    for address in tqdm(unique_addresses):
        full_response = search_district(address)
        file.write('{0}|{1}\n'.format(address,full_response))